In [ ]:
import pandas as pd
from collections import defaultdict

# --------------------------------------------
# SAMPLE BOM DATA (parent-child relationships)
# --------------------------------------------
data = [
    ["A", "B", 2, 5.00],
    ["A", "C", 1, 10.00],
    ["B", "D", 3, 1.00],
    ["B", "E", 2, 0.50],
]

bom_df = pd.DataFrame(data, columns=["Parent", "Child", "Qty", "Cost"])
print("\n=== Raw BOM ===")
print(bom_df)


# --------------------------------------------
# BUILD BOM STRUCTURE (tree/dict form)
# --------------------------------------------
def build_bom_tree(df):
    tree = defaultdict(list)
    for _, row in df.iterrows():
        tree[row["Parent"]].append({
            "child": row["Child"],
            "qty": row["Qty"],
            "cost": row["Cost"]
        })
    return tree


bom_tree = build_bom_tree(bom_df)


# --------------------------------------------
# EXPLODE THE BOM (get all lower components)
# --------------------------------------------
def explode_bom(tree, item, multiplier=1):
    result = []

    if item not in tree:
        return result  # leaf component

    for c in tree[item]:
        total_qty = c["qty"] * multiplier
        result.append({
            "parent": item,
            "child": c["child"],
            "qty_required": total_qty,
            "unit_cost": c["cost"],
            "total_cost": total_qty * c["cost"]
        })
        # recursion for multi-level structures
        result.extend(explode_bom(tree, c["child"], total_qty))
    return result


exploded = explode_bom(bom_tree, "A")
exploded_df = pd.DataFrame(exploded)

print("\n=== Exploded BOM for A ===")
print(exploded_df)


# --------------------------------------------
# COST ROLL-UP (total cost of top-level item)
# --------------------------------------------
def rollup_cost(exploded_df):
    return exploded_df["total_cost"].sum()

total_cost_A = rollup_cost(exploded_df)

print(f"\nTotal Manufacturing Cost of A = ${total_cost_A:.2f}")


# --------------------------------------------
# SUMMARIZED COMPONENT REQUIREMENTS
# --------------------------------------------
def summarize_requirements(exploded_df):
    summary = exploded_df.groupby("child") \
        .agg(total_qty=("qty_required", "sum"),
             unit_cost=("unit_cost", "first"),
             cost=("total_cost", "sum")) \
        .reset_index()
    return summary


summary_df = summarize_requirements(exploded_df)

print("\n=== Summary of Required Components ===")
print(summary_df)


In [ ]:
A → B
A → C
B → D
B → E


In [ ]:
Total Manufacturing Cost of A = $??.??  
